다음 강의는 [DeepLearningAI의 Multi AI Agent Systems with crewAI](https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/4/create-agents-to-research-and-write-an-article-(code)) 입니다.

# L2: 연구 및 기사 작성을 위한 에이전트 생성

이제 연구 및 기사 작성을 위한 에이전트를 생성해 보겠습니다. 이 에이전트들은 주제에 대한 정보를 수집하고 분석하여 풍부하고 흥미로운 기사를 작성하는 데 도움이 될 것입니다. 에이전트들은 다양한 소스에서 정보를 수집하고 필요한 내용을 요약하여 제공할 수 있습니다. 또한 에이전트들은 특정 주제에 대한 전문 지식을 가지고 있어 정확하고 신뢰할 수 있는 정보를 제공할 수 있습니다.

에이전트들은 자연어 처리 및 기계 학습 기술을 사용하여 효율적으로 정보를 수집하고 분석할 수 있습니다. 또한 에이전트들은 사용자의 요구에 맞게 커스터마이징되어 특정 주제나 관심사에 대한 정보를 제공할 수 있습니다.

이제 연구 및 기사 작성을 위한 에이전트를 생성하여 효율적이고 효과적인 작업을 수행할 수 있도록 해보세요.안녕하세요! 무엇을 도와드릴까요?이 강의에서는 다중 에이전트 시스템의 기본 개념을 소개하고 crewAI 프레임워크에 대한 개요를 얻게 될 것입니다.

 만약 여러분이 자신의 컴퓨터에서 이 노트북을 실행 중이라면,

다음 명령어를 사용하여 crewai, crewai_tools 및 langchain_community 패키지를 설치할 수 있습니다.

```
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- 크루AI 라이브러리에서 가져오기.

In [3]:
from crewai import Agent, Task, Crew

- 에이전트들을 위한 LLM으로 OpenAI의 `gpt-3.5-turbo`를 사용하게 될 것입니다.

**참고 사항:** crewAI는 에이전트의 LLM으로 다른 인기있는 모델도 사용할 수 있습니다. 몇 가지 예시는 노트북의 [맨 아래](#1)에서 확인할 수 있습니다.

- CrewAI는 Agent, Task 를 각각 만들고 디테일하게 정의하여 연결 후, crew 객체에 모아서 실행하게 됩니다.
    - 각 Agent는 명확하게 자신의 역할대로, 맥락을 이해하며 업무 수행
    - 각 task는 디테일하게 요건을 명확화 하여 정의

![](./img/crew1.png)

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## 에이전트 생성

에이전트는 사용자와 상호작용하고 작업을 수행하는 프로그램입니다. 에이전트를 만들 때는 몇 가지 중요한 요소를 고려해야 합니다. 에이전트의 목적과 기능을 명확히 이해하고, 사용자와의 상호작용 방식을 결정해야 합니다.

에이전트를 만들 때 고려해야 할 몇 가지 중요한 단계은 다음과 같습니다:

1. **목적 설정**: 에이전트가 수행해야 하는 작업과 목표를 명확히 정의합니다.
2. **상호작용 방식 결정**: 에이전트가 사용자와 어떻게 상호작용할지 결정합니다. 이는 대화형 인터페이스, 텍스트 기반 인터페이스, 음성 인식 등 다양한 형태로 이루어질 수 있습니다.
3. **기술 선택**: 에이전트를 구현하기 위해 어떤 기술을 사용할지 결정합니다. 이는 자연어 처리, 기계 학습, 강화 학습 등 다양한 기술을 포함할 수 있습니다.
4. **테스트 및 평가**: 에이전트를 테스트하고 성능을 평가하여 사용자 경험을 향상시키는 데 도움이 됩니다.



### Agent: Planner
참고 : 여러 문자열을 사용하는 이점은 다음과 같습니다.

<br><br>
varname = "텍스트의 1번째 줄"<br>
          "텍스트의 2번째 줄"<br>
<br>
세 개의 따옴표로 된 문서 문자열과 대조적으로
<br>

varname = """텍스트의 1번째 줄<br>
             텍스트의 2번째 줄<br>
          """<br><br>

여러 문자열을 사용하면 공백과 줄 바꿈 문자를 추가하지 않아 LLM에 전달하기에 더 나은 형식으로 만들 수 있습니다.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."
              "You are good at Korean language.",
    allow_delegation=False,
	verbose=True
)

안녕하세요! 어떤 도움이 필요하신가요?

### Agent:Writer

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "You are good at Korean language.",
    allow_delegation=False,
    verbose=True
)

안녕하세요! 어떻게 도와드릴까요?

### Agent:Editor

In [15]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible."
              "You are good at Korean language.",
    allow_delegation=False,
    verbose=True
)

## 작업 생성

작업을 만들 때는 명확하고 구체적인 목표를 설정하는 것이 중요합니다. 작업을 생성할 때 고려해야 할 몇 가지 중요한 단계가 있습니다.

1. **목표 설정**: 작업의 목표를 명확하게 정의하고 어떤 결과물을 얻고자 하는지 결정합니다.

2. **작업 내용 정의**: 작업의 내용을 상세히 기술하여 무엇을 해야 하는지 명확하게 이해할 수 있도록 합니다.

3. **마감일 설정**: 작업을 완료해야 하는 기한을 설정하여 시간을 효율적으로 관리합니다.

4. **우선순위 설정**: 작업의 중요도에 따라 우선순위를 정하고 중요한 작업을 먼저 처리합니다.

5. **할당 및 추적**: 작업을 담당자에게 할당하고 진행 상황을 추적하여 작업이 원활하게 진행되도록 합니다.

작업을 만들 때 이러한 단계를 따르면 작업을 효율적으로 관리하고 목표를 달성하는 데 도움이 됩니다.Hello! How can I assist you today?**Task 1: Data Entry**

- **Description:** Enter customer information into the database.
- **Expected Output:** Accurate and complete customer profiles in the database.
- **Agent:** Data entry specialist.

**Task 2: Customer Support**

- **Description:** Respond to customer inquiries and provide assistance.
- **Expected Output:** Satisfied customers with their questions answered and issues resolved.
- **Agent:** Customer support representative.

**Task 3: Social Media Management**

- **Description:** Create and schedule posts on social media platforms.
- **Expected Output:** Engaging and consistent social media presence.
- **Agent:** Social media manager.

**Task 4: Sales Calls**

- **Description:** Make outbound sales calls to potential customers.
- **Expected Output:** Increased sales and customer acquisition.
- **Agent:** Sales representative.

### 작업: 계획

1. 문제를 분석하고 이해한다.
2. 해결책을 찾기 위해 가능한 방법을 고려한다.
3. 해결책을 구현하기 위한 단계별 계획을 세운다.
4. 계획을 실행하고 문제를 해결한다.
5. 결과를 평가하고 필요한 조정을 한다.
6. 성공적으로 해결된 문제에 대한 배운 점을 기록하고 다음 비슷한 문제에 대비한다.

### Task: Plan

In [12]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources."
        "in Korean" ,
    agent=planner,
)

Hello, how can I help you today?

### Task: Write

In [13]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs."
        "in Korean",
    agent=writer,
)

I am ready to help with editing. Please provide the text that needs editing.

### Task: edit

In [16]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs."
                    "in Korean",
    agent=editor
)

## Creating the Crew

CrewAI 객체 생성 


In [17]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew


**참고**: LLMs는 동일한 입력에 대해 다른 출력을 제공할 수 있으므로, 여러분이 보는 것과 다를 수 있습니다.

In [18]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

콘텐츠 계획서: 인공지능에 대한 포괄적인 내용 계획서

개요:
- 최신 트렌드, 핵심 플레이어 및 주목할만한 뉴스를 중점적으로 다룰 것
- 타겟 오디언스를 식별할 것이며, 그들의 관심사와 고통 포인트를 고려할 것
- 소개, 주요 포인트 및 행동 요구 사항을 포함한 자세한 내용 개요를 개발할 것
- SEO 키워드와 관련 데이터 또는 소스를 포함할 것

대상 오디언스:
- 기술 업계 전문가 및 학생
- 비즈니스 리더 및 기업주
- 일반 대중

키워드:
- 인공지능, 딥러닝, 기계 학습, 자연어 처리, 로봇 공학

내용 개요:
I. 소개
- 인공지능의 개념 소개
- 인공지능의 중요성 강조

II. 최신 트렌드
- 최근의 인공지능 기술 및 응용 분야 소개
- 주요 기업의 인공지능 관련 뉴스 소개

III. 핵심 플레이어
- 주요 인공지능 기업 및 연구소 소개
- 현재의 인공지능 시장 동향 분석

IV. 타겟 오디언스의 관심사
- 기술 업계 전문가의 관심사 및 요구 사항
- 비즈니스 리더의 인공지능 활용에 대한 고

In [20]:
from IPython.display import Markdown
Markdown(result)

# **인공지능: 현대 기술의 미래를 이끄는 힘**

## **소개**
인공지능은 인간의 지능을 모방하거나 대체하기 위한 컴퓨터 시스템과 관련 기술을 의미합니다. 이러한 기술은 우리의 삶과 사회 전반에 혁명적인 변화를 가져오고 있습니다. 기계 학습, 딥 러닝, 자연어 처리, 로봇 공학 등 다양한 분야에서 활용되고 있으며, 빠르게 발전하고 있습니다.

인공지능 기술의 발전 속도와 윤리적 문제에 대한 기술 업계 전문가들의 관심은 더욱 높아지고 있습니다. 또한, 비즈니스 리더들은 인공지능을 활용하여 생산성을 향상시키고 비용을 절감하려는 고통 포인트를 겪고 있습니다.

## **최신 트렌드**
현재, 인공지능 기술은 의학, 금융, 자동차, 소매업 등 다양한 산업에 적용되어 있습니다. 자율 주행 자동차, 의료 진단 보조 시스템, 언어 번역 프로그램 등이 그 예시입니다. 또한, 구글, 아마존, 마이크로소프트와 같은 주요 기업들이 인공지능 기술을 활발히 연구하고 있습니다.

인공지능 분야의 핵심 기업으로는 구글의 딥마인드, 오픈AI, IBM 왓슨 등이 있습니다. 이들은 혁신적인 기술과 솔루션을 제공하며, 인공지능 시장을 선도하고 있습니다.

## **타겟 오디언스의 관심사**
기술 업계 전문가들은 인공지능 기술의 발전 속도와 윤리적 문제에 대해 관심을 갖고 있습니다. 비즈니스 리더들은 인공지능을 활용하여 생산성을 향상시키고 비용을 절감하려는 고통 포인트를 겪고 있습니다.

이 블로그 포스트는 '인공지능', '딥러닝', '기계 학습', '자연어 처리', '로봇 공학'과 같은 키워드를 자연스럽게 삽입하여 검색 엔진 최적화를 진행하였습니다.

더 많은 정보를 원하시는 분들은 Gartner, Forrester와 같은 시장 조사 보고서를 참고하거나 TensorFlow, OpenAI와 같은 인공지능 관련 플랫폼을 살펴보시기 바랍니다.

인공지능은 우리의 미래를 형성하는 기술 중 하나로, 그 발전은 빠르고 놀라운 결과를 가져오고 있습니다. 우리는 이 기술을 올바르게 활용하여 사회적 이슈를 고려하고 더 나은 미래를 구축해야 합니다.

In [21]:
topic = "40대에 꼭 해야할 10가지 일"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on 40대에 꼭 해야할 10가지 일.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

컨텐츠 계획서

제목: 40대에 꼭 해야할 10가지 일

개요: 본 블로그 기사는 40대에게 꼭 해야할 10가지 일에 대해 다뤄질 예정입니다. 최신 트렌드, 핵심 인물 및 주목할만한 뉴스를 우선적으로 파악하여 타겟 오디언스의 이해와 관심사, 고통 포인트를 고려한 상세한 컨텐츠 아웃라인을 개발할 것입니다. SEO 키워드를 적용하고 관련 데이터나 소스를 추가하여 정보를 제공할 것입니다.

아웃라인:
1. 서론
- 40대에게 중요한 것은 무엇인가?
- 왜 40대에게 해야할 일이 중요한가?
2. 40대에게 꼭 해야할 10가지 일 소개
- 건강 관리
- 가족 시간 보내기
- 재테크
- 자기계발
- 여가 활동
- 사회 공헌
- 취미 개발
- 스트레스 관리
- 여행
- 친구 관계 유지
3. 마무리와 행동 요령
- 40대들에게 권고하는 행동 요령
- 더 많은 정보를 얻을 수 있는 방법
- 더 나아가기 위한 다음 단계

타겟 오디언스 분석:
- 40대의 관심사: 건강, 가족, 자기계발, 여가 활동, 재테크 등
- 40대의 고통 포인

In [22]:
Markdown(result)

# 40대에 꼭 해야할 10가지 일

## 서론
40대에게 중요한 것은 무엇인가? 40대는 가족, 건강, 자기계발, 여가 활동, 그리고 재테크와 같은 다양한 측면에서 관심을 갖고 있습니다. 이들은 이제 자신의 삶에 대해 깊이 생각하고, 더 나은 미래를 위해 행동하려는 단계에 있습니다. 따라서 40대에게 해야할 일은 그들이 균형 잡힌 삶을 살 수 있도록 돕는 것이 중요합니다.

왜 40대에게 해야할 일이 중요한가? 40대는 이미 중년에 접어들었고, 이제부터 자신의 건강과 가치관, 가족과의 관계, 그리고 미래를 위한 준비 등을 심각하게 고민해야 합니다. 이 시기에 적절한 행동을 취하지 않으면 미래에 후회할 수 있는 상황이 발생할 수 있기 때문에 40대에게 해야할 일은 그들의 삶을 더욱 풍요롭고 만족스럽게 만들어줄 것입니다.

## 40대에게 꼭 해야할 10가지 일 소개

### 건강 관리
40대에게 가장 중요한 것은 건강입니다. 건강한 신체는 건강한 정신을 낳으며, 이를 통해 삶의 질을 향상시킬 수 있습니다. 꾸준한 운동과 균형 잡힌 식사, 그리고 정기적인 건강 검진은 건강한 노후를 위한 필수 요소입니다.

### 가족 시간 보내기
가족은 40대에게 큰 의미를 갖습니다. 가족과의 소중한 시간을 보내고 소통하며 서로의 지지를 받는 것은 행복한 가정을 유지하는 데 중요합니다. 주말에 가족들과 함께하는 야외 활동이나 가족 모임을 통해 가족 간 유대감을 증진시키는 것이 좋습니다.

### 재테크
재테크는 40대에게 꼭 필요한 일 중 하나입니다. 미래를 위한 자산을 쌓고 효율적으로 관리하는 것은 안정적인 노후를 보장하는 데 도움이 됩니다. 퇴직 연금, 주식 투자, 부동산 투자 등 다양한 재테크 방법을 고려해보는 것이 좋습니다.

### 자기계발
40대에게 자기계발은 더 나은 인생을 살기 위한 핵심 요소입니다. 새로운 취미나 관심사를 발견하거나 다양한 스킬을 배우는 것은 삶의 만족도를 높이는 데 도움이 됩니다. 독서, 온라인 강의 수강, 현지 커뮤니티 활동 등을 통해 자기계발에 힘써보는 것이 좋습니다.

### 여가 활동
스트레스 해소와 휴식을 위한 여가 활동은 40대에게 꼭 필요합니다. 일상의 모노톤한 일상을 벗어나 즐거움을 느낄 수 있는 취미나 레저 활동을 통해 마음의 여유를 찾는 것이 중요합니다. 등산, 요가, 그림 그리기 등 다양한 여가 활동을 통해 자신만의 행복을 찾아보는 것이 좋습니다.

### 사회 공헌
사회 공헌은 40대에게 꼭 필요한 가치 중 하나입니다. 자신의 일상을 넘어 타인을 돕고 선한 영향력을 미치는 것은 보람 있는 삶을 살 수 있는 길입니다. 자원봉사, 기부 활동, 지역 사회 활동 등을 통해 사회적 가치를 실현하는 것이 좋습니다.

### 취미 개발
취미는 40대에게 삶에 활력을 불어넣어줄 수 있는 중요한 요소입니다. 자신만의 취미를 발견하고 이를 통해 스트레스를 해소하고 행복을 찾아보는 것이 좋습니다. 요리, 바둑, 피아노 연주 등 다양한 취미를 통해 자신만의 특별한 시간을 만들어보는 것이 좋습니다.

### 스트레스 관리
스트레스는 40대에게 피할 수 없는 현실입니다. 하지만 적절한 스트레스 관리를 통해 스트레스를 효과적으로 해소할 수 있습니다. 정기적인 운동, 명상, 호흡법 등을 통해 스트레스를 관리하고 자신의 마음을 안정시키는 것이 중요합니다.

### 여행
여행은 40대에게 새로운 자극과 경험을 제공해줄 수 있는 좋은 방법입니다. 여행을 통해 새로운 문화를 경험하고 새로운 사람들을 만나는 것은 삶에 활력을 불어넣어줄 수 있습니다. 근교 나들이부터 해외 여행까지 다양한 여행을 즐기며 자신만의 추억을 쌓아보는 것이 좋습니다.

### 친구 관계 유지
친구 관계는 40대에게 소중한 지원체계입니다. 오랜 친구들과 소통을 유지하고 새로운 인연을 만나는 것은 삶의 품격을 높이는 데 도움이 됩니다. 정기적인 만남이나 소규모 소셜 네트워킹을 통해 친구 관계를 유지하고 소중한 인연을 지속하는 것이 좋습니다.

## 마무리와 행동 요령
40대들에게 권고하는 행동 요령은 다양한 측면에서의 균형 잡힌 삶을 살아가는 것입니다. 건강 관리, 자기계발, 가족 시간 보내기, 재테크 등을 꾸준히 실천하여 안정적이고 행복한 삶을 살아가는 것이 중요합니다. 또한 더 많은 정보를 얻기 위해 관련 자료와 소스를 참고하고, 더 나아가기 위한 다음 단계를 고민하는 것도 중요합니다.

이상으로 40대에게 꼭 해야할 10가지 일에 대해 소개했습니다. 40대 여러분께서는 이를 참고하여 더 나은 미래를 위한 다양한 행동을 취해보시기 바랍니다. 삶의 다양한 영역에서 균형을 유지하고 풍요로운 삶을 즐기는 것이 중요합니다. 함께 건강하고 행복한 40대를 위해 노력해봅시다.

---
SEO Keywords: 40대, 해야할 일, 건강, 가족, 재테크, 자기계발, 여가, 스트레스, 여행, 친구 관계

Related Sources:
- 보건복지부 보도자료
- 한국가정경제학회 논문
- 통계청 조사 결과

이제 여러분은 40대에게 꼭 해야할 10가지 일에 대해 더 깊이 생각해보고, 실천해보는 것이 중요합니다. 행복하고 풍요로운 40대를 위해 노력해봅시다. 감사합니다.

## 다른 인기 있는 모델: LLM을 위한 에이전트들

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```



#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

로컬에서 Ollama 및 기타 Llama와 함께 Llama를 사용하려면 [모든 LLM에 연결하는 방법](https://docs.crewai.com/how-to/LLM-Connections/)에 대한 crewAI 문서를 확인해보세요.